In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hw1_2 import *
from torch.optim import Adam,SGD
from torch.autograd import Variable
from tqdm import tqdm
import datetime

In [3]:
fcn = myFcn16()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(fcn.parameters(), lr=0.0002)

train_loader = loadData('hw1_data/hw1_data/p2_data/train',data_aug=True)
val_loader = loadData('hw1_data/hw1_data/p2_data/validation')

In [4]:
def saveModel(name=''):
    path = "./hw1_2_fcn16"+name+".pth"
    torch.save(fcn.state_dict(), path)

def val():
    iou_score = 0
    val_loss = []
    fcn.eval().cuda()
    tp_fn = torch.zeros(6).cuda()
    tp_fp = torch.zeros(6).cuda()
    tp = torch.zeros(6).cuda()
    
    with torch.no_grad():
        for k, val_data in enumerate(val_loader):
            val_img, val_label = val_data
            
            val_img = val_img.cuda()
            val_label = val_label.cuda()
            
            pred = fcn(val_img)
            los = criterion(pred,val_label.long())
            val_loss.append(los)
            
            pred = torch.argmax(pred,dim=1)
            
            for i in range(6):
                tp_fp[i] += torch.sum(pred == i)
                tp_fn[i] += torch.sum(val_label == i)
                tp[i] += torch.sum( (pred==i) * (val_label == i))
    
    val_loss = sum(val_loss) / len(val_loss)
    for i in range(6):
        iou = tp[i] / (tp_fp[i]+tp_fn[i]-tp[i])
        iou_score += iou/6
        
    print("miou: ",round(iou_score.item(),3))
    return val_loss
            

In [5]:
best_loss = 10000000000
for epoch in range(60):
    fcn.train()
    
    for dat in tqdm(train_loader):
        inputs, labels = dat
        
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = fcn(inputs)
        loss = criterion(outputs,labels.long())
        
        loss.backward()
        optimizer.step()
        
    print('Epoch',epoch,'\nTrain loss:',round(loss.item(),2))
    val_loss = val()
    if(val_loss < best_loss):
        best_loss = val_loss
        saveModel('Best')
    if(epoch==1):
        saveModel('Start')
    if(epoch==30):
        saveModel('Middle')
    if(epoch%5==4):
        saveModel('Tmp')
        print("Saving Model.....")
saveModel('End')
print("Done Training")

100%|███████████████████████████████████████████████████| 667/667 [08:59<00:00,  1.24it/s]


Epoch 0 
Train loss: 0.66
miou:  0.574


100%|███████████████████████████████████████████████████| 667/667 [08:52<00:00,  1.25it/s]


Epoch 1 
Train loss: 0.59
miou:  0.625


100%|███████████████████████████████████████████████████| 667/667 [08:52<00:00,  1.25it/s]


...

Epoch 57 
Train loss: 0.02
miou:  0.736


100%|███████████████████████████████████████████████████| 667/667 [08:51<00:00,  1.26it/s]


Epoch 58 
Train loss: 0.03
miou:  0.734


100%|███████████████████████████████████████████████████| 667/667 [08:57<00:00,  1.24it/s]


Epoch 59 
Train loss: 0.03
miou:  0.736
Saving Model.....
Done Training
